# Логистическая регрессия. Практическая работа

## Цель практической работы

Вы уже делали задание, в котором с помощью метода ближайших соседей оценивали склонность клиента банка откликнуться или не откликнуться на предложение.

В этом практическом задании ваши цели:
*  решить эту же задачу с помощью логистической регрессии;
*  потренироваться в подборе порога; 
*  потренироваться в подборе гиперпараметров модели.

## Что входит в работу

*  Загрузить данные для задачи.
*  Обучить метод ближайших соседей с заданным количеством соседей k, вычислить метрики.
*  Обучить логистическую регрессию с параметрами по умолчанию, вычислить метрики.
*  Подобрать порог модели, вычислить метрики.
*  Подобрать гиперпараметр С (константа регуляризации) модели, вычислить метрики.

## Что оценивается

*  Выполнены все этапы задания: код запускается, отрабатывает без ошибок; подробно и обоснованно написаны текстовые выводы, где это требуется.

## Формат сдачи
Выполните предложенные задания — впишите свой код (или, если требуется, текст) в ячейки после комментариев. 

*Комментарии — это текст, который начинается с символа #. Например: # ваш код здесь.*

Сохраните изменения, используя опцию Save and Checkpoint из вкладки меню File или кнопку Save and Checkpoint на панели инструментов. Итоговый файл в формате .ipynb (файл Jupyter Notebook) загрузите в личный кабинет и отправьте на проверку.

In [1]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sklearn

In [3]:
# считаем данные
data = pd.read_csv('D:/STUDY/ml_skill_box/data/ClientsData.csv') #D:\STUDY\ml_skill_box

In [4]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


В этом ноутбуке нам придётся подбирать гиперпараметры модели, а ещё порог. Поэтому, чтобы не переобучиться, разобъём данные на трейн, валидацию и тест.

*  Обучать модели будем на тренировочных данных.
*  Подбирать необходимые величины — по валидации.
*  Оценивать качество — на тесте.

In [17]:
# разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

В задании по методу ближайших соседей было найдено, что оптимальное число соседей k = 9.

Обучите на тренировочных данных KNN с k = 9 и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [7]:
def print_metric(y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(f"Precision = {precision_score(y_test, y_pred)}")
    print(f"Recall = {recall_score(y_test, y_pred)}")

In [8]:
# ваш код здесь
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(Xtrain, ytrain)
ypred = knn.predict(Xval)
print_metric(yval, ypred)

[[2827   17]
 [ 348    5]]
Precision = 0.22727272727272727
Recall = 0.014164305949008499


Какой вывод можно сделать:
- для класса 0 — клиент не откликнулся — мы получили достаточно высокие значения TP в том числе потому, что представителей этого класса больше;
- для класса 1 — клиент откликнулся — мы получили низкие значения TN.

Поэтому значения precision и recall низкие. Модель даёт неудовлетворительные результаты, так как находит мало клиентов, которые откликнутся на предложение.



Обучите логистическую регрессию с параметрами по умолчанию и посмотрите на метрики.

Везде дальше при оценке метрик надо выводить confusion_matrix, precision и recall.

In [10]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь
log_reg = LogisticRegression().fit(Xtrain, ytrain)
ypred = log_reg.predict(Xval)
print_metric(yval, ypred)

[[2841    3]
 [ 353    0]]
Precision = 0.0
Recall = 0.0


Наша цель — найти как можно больше клиентов, которые откликнутся на предложение. А модель таких не находит. 

Мы помним, что метод predict_proba у логистической регрессии предсказывает математические (то есть корректные) вероятности классов. Предскажите вероятности классов с помощью обученной логистической регрессии на тестовых данных и выведите вероятности положительного класса для первых десяти объектов. 

Глядя на полученные вероятности, попробуйте объяснить, почему вы получили именно такую матрицу ошибок и такие значения точности с полноты.

In [12]:
# ваш код здесь
ypred_proba = log_reg.predict_proba(Xval)
ypred_proba[:, 1][:10] >

array([False, False, False, False, False, False, False, False, False,
       False])

In [ ]:
# ваше объяснение здесь
# Мы получили такую матрицу ошибки, так как predict_proba выдает крайне низкие вероятности

Давайте уточним цель. Пусть нам нужно найти как можно больше клиентов, которые откликнутся на предложение, то есть максимизировать полноту (recall). 

При этом хочется, чтобы точность модели (precision) не была очень низкой. Заказчик посмотрел на результаты работы KNN и потребовал, чтобы точность была не ниже, чем у KNN: $precision \geq 0.13$.

Давайте будем изменять порог для перевода вероятности в классы так, чтобы:
*   максимизировать значение recall
*   при условии, что $precision \geq 0.13$.

Если мы будем подбирать порог по тестовой выборке, то, по сути, обучимся на ней и, значит, переобучимся. Это плохо. 

Поэтому предскажите вероятности на валидационной выборке и подберите порог по ней (Xval, yval), а затем посмотрите, какое качество для найденного порога вы получите на тестовых данных.

In [29]:
probs_val = log_reg.predict_proba(Xval)

max_recall = -1
thr = -1
prec = -1

for threshold in np.arange(0.05, 1, 0.001):
    # для каждого значения порога переведите вероятности в классы
    # посчитайте метрики
    yval_pred = probs_val[:, 1] >= threshold
    precision = precision_score(yval, yval_pred)
    recall = recall_score(yval, yval_pred)
    if precision >= 0.13:
        if recall > max_recall:
            thr = threshold
            max_recall = recall
# напечатайте порог, для которого получается максимальная полнота, при precision >= 0.13

In [31]:
# по найденному порогу переведите вероятности в классы на тесте и напечатайте метрики

y_test_pred = log_reg.predict_proba(Xtest)[:, 1] >= thr
print_metric(ytest, y_test_pred)

[[1061 2947]
 [  88  471]]
Precision = 0.13779988297249854
Recall = 0.8425760286225402


Сделайте вывод. Смогли ли мы с помощью подбора порога добиться большего значения recall, чем у KNN? 

In [22]:
# ваш вывод здесь
# Precison стал меньше, но recall cтал больше


А ещё, чтобы улучшить качество предсказания, можно подбирать значение гиперпараметра C у логистической регрессии. Для каждого значения C придётся подбирать свой порог, поэтому:  

1. Обучите для значений C из диапазона [0.05, 0.15, 0.25, ...., 10.05] логистическую регрессию (на тренировочных данных).

2. Для каждой из обученных моделей во внутреннем цикле подберите оптимальный порог (как в предыдущем задании) — на валидационных данных.



В качестве результата выведите значение C и порога для модели, которая даёт наилучшие значения метрик (наибольший recall при ограничении на $precision \geq 0.13$).

Также напечатайте полученные метрики (матрицу ошибок, точность и полноту) для лучшей модели — на тестовых данных.

In [32]:
regs = []
recalls = []
thresholds = []
precisions = []

for reg in np.arange(0.001, 1, 0.01):
    lr = LogisticRegression(C=reg)
    lr.fit(Xtrain, ytrain)
    
    y_pred_proba = lr.predict_proba(Xval)[:, 1]
    
    max_recall = -1
    thr = -1
    prec = -1
    
    for threshold in np.arange(0.05, 0.25, 0.001):
        y_pred = (y_pred_proba >= threshold).astype(int)
        recall = recall_score(yval, y_pred)
        precision = precision_score(yval, y_pred)
        if precision >= 0.13:
            if recall > max_recall:
                max_recall = recall
                thr = threshold
                prec = precision
    
    recalls.append(max_recall)
    thresholds.append(thr)
    precisions.append(prec)
    regs.append(reg)

# находим индекс лучшей модели
best_idx = np.argmax(recalls)


In [33]:
# выведите значения C, precision, recall, threshold для наилучшей по заданным условиям модели
print("Best C:", regs[best_idx])
print("Best threshold:", thresholds[best_idx])
print("Best precision:", precisions[best_idx])
print("Best recall:", recalls[best_idx])

Best C: 0.28099999999999997
Best threshold: 0.07100000000000002
Best precision: 0.1301751592356688
Best recall: 0.9263456090651558


In [28]:
# с помощью найденных C и threshold обучите модель на тренировочных данных, сделайте предсказание на тесте и по найденному порогу получите классы
# напечатайте метрики

lr = LogisticRegression(C=regs[best_idx])
lr.fit(Xtrain, ytrain)

y_pred_proba = lr.predict_proba(Xtest)[:, 1]
y_pred = (y_pred_proba >= thresholds[best_idx]).astype(int)

# Печатаем матрицу ошибок, точность и полноту на тестовом наборе
print("Матрица ошибок:")
print_metric(ytest, y_pred)


Матрица ошибок:
[[ 905 3103]
 [  70  489]]
Precision = 0.1361358574610245
Recall = 0.8747763864042933


Влияет ли изменение гиперпараметра C на качество модели (и, соответственно, метрики) в этой задаче?

С помощью подборов гиперпараметров мы смогли повысить качество модели.

Ответьте развёрнуто на следующие вопросы:

* Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче? 

* Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?


1. Recal у лог. регрессия выше, но accuracy у knn выше.
2. Да